# Examen 2 - Sistemas de Recomendacion

##Roberth Ramos


## Pregunta 1
 Explica como el algoritmo k-nearest neighbors KNN utiliza metricas de similitud para generar recomendaciones, indica dos ventajas y dos limitaciones del uso de knn en un sistema de recomentacion con datos de peliculas.

** Respuesta **
 En un sistema de recomendación de películas, KNN utiliza métricas de similitud, como la distancia euclidiana o el coseno de similitud o correlacion de pearson, para encontrar películas similares a las que un usuario ha visto o calificado previamente. Esto lo ejecuta en 3 pasos:
 - Calcula Metrica de similitud, calcula la similitud utilizando algunas de las metricas como distancia euclidiana, similitud de coseno, etc.
 - Identificacion de vecimos, el algoritmo encuentra encuentra las peliculas las cercanas de acuerdo a la metrica de similitud utilizada.
 - Recomendación, las películas que resultan ser los "vecinos más cercanos" de las ya vistas o calificadas por el usuario son recomendadas como películas que probablemente le gustarán.

 Ventajas
 - Facil de entender y explicar
 - No requieren entrenamiento previo
 Desventajas
 - datos dispersos afectan el modelo
 - escalabilidad.


## Pregunta 2

In [2]:
# Instalar la librería scikit-surprise para algoritmos de filtrado colaborativo
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357291 sha256=a7747af9e3f7111f35ee21472c60e672001409f1de420a014e426aa5b5276f2b
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [3]:
# Descargar el dataset MovieLens 1M
!curl -o dataset.zip "https://files.grouplens.org/datasets/movielens/ml-1m.zip"
!unzip dataset.zip
!ls -la

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 5778k  100 5778k    0     0  9541k      0 --:--:-- --:--:-- --:--:-- 9536k
Archive:  dataset.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         
total 5800
drwxr-xr-x 1 root root    4096 Oct 29 00:47 .
drwxr-xr-x 1 root root    4096 Oct 29 00:27 ..
drwxr-xr-x 4 root root    4096 Oct 25 13:20 .config
-rw-r--r-- 1 root root 5917549 Oct 29 00:47 dataset.zip
drwxr-x--- 2 root root    4096 Jan 29  2016 ml-1m
drwxr-xr-x 1 root root    4096 Oct 25 13:20 sample_data


In [5]:
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import KNNBasic
import pandas as pd
import random

In [6]:
# Cargar los datasets de calificaciones y películas
ratings = pd.read_csv('ml-1m/ratings.dat', sep='::', header=None, engine='python',
                      names=['userId', 'movieId', 'rating', 'timestamp'], encoding='latin-1')
movies  = pd.read_csv('ml-1m/movies.dat', sep='::',  header=None, engine='python',
                      names=['movieId', 'title', 'genres'], encoding='latin-1')


In [7]:
# Unir los datasets en base a 'movieId' para agregar los títulos
user_item_rating = pd.merge(ratings, movies[['movieId', 'title']], on='movieId')

# Seleccionar solo las columnas necesarias
user_item_rating = user_item_rating[['userId', 'title', 'rating']]

# Ordenar los datos por 'userId'
user_item_rating.sort_values(by='userId', inplace=True)

# Mostrar las primeras filas para verificar
user_item_rating.head()

,userId,title,rating
0,1,One Flew Over the Cuckoo's Nest (1975),5
29,1,"Close Shave, A (1995)",3
30,1,Antz (1998),4
31,1,"Girl, Interrupted (1999)",4
32,1,Hercules (1997),4


In [8]:
# Definir el formato de los datos para scikit-surprise
reader = Reader(rating_scale=(1, 5))

# Crear el dataset desde el DataFrame
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)


In [9]:
# Separar los datos en entrenamiento y prueba
trainset, testset = train_test_split(data, test_size=0.25)

# Usar el algoritmo KNN con la similitud de Pearson para encontrar la similitud entre usuarios
sim_options = {
    'name': 'pearson',  # Usamos correlación de Pearson
    'user_based': True  # Similitud basada en usuarios
}
algo = KNNBasic(sim_options=sim_options)

In [10]:

# Entrenar el algoritmo
algo.fit(trainset)


Computing the pearson similarity matrix...
Done computing similarity matrix.


In [11]:
# Seleccionamos dos usuarios al azar que hayan valorado al menos 5 películas en común
user_ids = ratings['userId'].value_counts()
eligible_users = user_ids[user_ids >= 5].index.tolist()

In [16]:
# Seleccionar dos usuarios al azar
user1, user2 = random.sample(eligible_users, 2)

# Obtener la similitud entre los dos usuarios
sim_matrix = algo.sim

# Convertir los IDs de usuarios a los índices de scikit-surprise
user1_inner_id = algo.trainset.to_inner_uid(user1)
user2_inner_id = algo.trainset.to_inner_uid(user2)

# Extraer la similitud entre los usuarios
similarity_score = sim_matrix[user1_inner_id][user2_inner_id]

# Imprimir los resultados
print(f"Los usuarios seleccionados son: {user1} y {user2}")
print(f"La similitud (correlación de Pearson) entre los usuarios es: {similarity_score}")

Los usuarios seleccionados son: 2219 y 4500
La similitud (correlación de Pearson) entre los usuarios es: 0.9847319278346618


Conclusion:

La correlación de Pearson entre los usuarios seleccionados (2219 y 4500) es 0.98, lo que indica una muy alta similitud entre las calificaciones que estos usuarios han dado a las películas que han visto en común. Este valor está muy cerca de 1, lo que sugiere que estos usuarios tienen gustos extremadamente similares.

Interpretación del resultado:

Alta correlación (0.98): Esto significa que los usuarios 2219 y 4500 tienden a calificar las películas de manera muy similar. Si a uno de estos usuarios le gusta o no una película, es muy probable que el otro usuario también tenga una opinión similar sobre esa misma película.

Implicaciones para el sistema de recomendación:

Recomendación entre usuarios: En un sistema de recomendación colaborativa basado en usuarios, esto indica que las películas que ha visto uno de estos usuarios pueden ser altamente recomendables para el otro. El sistema puede confiar en que las preferencias de uno reflejan bastante bien las del otro.
Eficiencia del modelo: Este tipo de correlación alta sugiere que estos usuarios podrían formar parte de un mismo grupo de "usuarios similares", y las recomendaciones de películas para ellos serían bastante precisas.

## Pregunta 3

Escoge una pelicula que tenga al menos 50 valoraciones y calcula con otras dos peliculas del dataset usando similitud de coseno. Explica como podrías usar estos resultados para sugerir recomendaciones basadas en items

In [17]:


# Usar el algoritmo KNN con la similitud de coseno para calcular similitudes entre ítems (películas)
sim_options = {
    'name': 'cosine',  # Usamos similitud de coseno
    'user_based': False  # Similitud basada en ítems (películas)
}
algo = KNNBasic(sim_options=sim_options)

# Entrenar el algoritmo
algo.fit(trainset)


Computing the cosine similarity matrix...
Done computing similarity matrix.


In [18]:

# Seleccionar una película con al menos 50 valoraciones
movie_counts = ratings['movieId'].value_counts()
selected_movie_id = movie_counts[movie_counts >= 50].index[0]

# Elegir otras dos películas para comparar (diferentes a la primera)
other_movie_ids = movie_counts[movie_counts >= 50].index[1:3]

# Convertir los IDs de películas a los índices de scikit-surprise
selected_movie_inner_id = algo.trainset.to_inner_iid(selected_movie_id)
other_movie_inner_ids = [algo.trainset.to_inner_iid(mid) for mid in other_movie_ids]

# Obtener la similitud de coseno entre la película seleccionada y las otras dos
similarity_with_first_movie = algo.sim[selected_movie_inner_id][other_movie_inner_ids[0]]
similarity_with_second_movie = algo.sim[selected_movie_inner_id][other_movie_inner_ids[1]]

# Obtener los títulos de las películas seleccionadas para mostrar los resultados
selected_movie_title = movies[movies['movieId'] == selected_movie_id]['title'].values[0]
other_movie_titles = movies[movies['movieId'].isin(other_movie_ids)]['title'].values

# Mostrar los resultados
print(f"Similitud de coseno entre '{selected_movie_title}' y '{other_movie_titles[0]}': {similarity_with_first_movie}")
print(f"Similitud de coseno entre '{selected_movie_title}' y '{other_movie_titles[1]}': {similarity_with_second_movie}")


Similitud de coseno entre 'American Beauty (1999)' y 'Star Wars: Episode IV - A New Hope (1977)': 0.9631651181371084
Similitud de coseno entre 'American Beauty (1999)' y 'Star Wars: Episode V - The Empire Strikes Back (1980)': 0.9621719818046678



Interpretación de los resultados:
Similitud de coseno entre 'American Beauty (1999)' y 'Star Wars: Episode IV - A New Hope (1977)': 0.96
Similitud de coseno entre 'American Beauty (1999)' y 'Star Wars: Episode V - The Empire Strikes Back (1980)': 0.96
Ambos valores de similitud de coseno son muy altos (0.96), lo que indica que las calificaciones que los usuarios han dado a estas películas son muy similares, aunque los géneros de las películas son claramente diferentes (drama vs. ciencia ficción).

¿Cómo podemos usar estos resultados para hacer recomendaciones basadas en ítems?
En un sistema de recomendaciones basado en ítems (películas), los resultados de similitud de coseno se pueden utilizar de varias formas:

1. Recomendaciones basadas en películas que el usuario ha disfrutado:
Si un usuario ha calificado positivamente una de estas películas, como 'American Beauty', el sistema puede recomendarle películas con alta similitud de coseno, como 'Star Wars: Episode IV' o 'Star Wars: Episode V', basándose en las preferencias de otros usuarios.
A pesar de que los géneros sean diferentes, la similitud en la forma en que los usuarios han calificado las películas indica que hay una fuerte correlación en las preferencias.
2. Filtrado por similitud de coseno:
El sistema puede priorizar recomendar películas con un valor de similitud de coseno alto (por ejemplo, por encima de 0.8 o 0.9), ya que las probabilidades de que el usuario disfrute de una película similar a una que ya le ha gustado son mayores.
En este caso, las películas de 'Star Wars' podrían ser buenas candidatas a ser recomendadas a alguien que disfrutó de 'American Beauty', debido a la fuerte similitud en las calificaciones de otros usuarios.
3. Mejorar la diversidad de recomendaciones:
Aunque las películas recomendadas sean de géneros distintos, como en este caso, la similitud de coseno sugiere que los usuarios que disfrutan de 'American Beauty' también podrían disfrutar de 'Star Wars', aportando diversidad a las recomendaciones.
Esto puede ser útil para descubrir nuevos géneros o estilos que un usuario podría no haber considerado previamente.
4. Sistema híbrido:
Un sistema que combine la similitud de ítems (películas) con información de géneros o contenido podría afinar mejor las recomendaciones. Por ejemplo, aunque la similitud de calificaciones entre 'American Beauty' y 'Star Wars' es alta, un filtro adicional podría limitar las recomendaciones a un conjunto de géneros que el usuario prefiera.




## Pregunta 4

Implementa un sistema de recomendacion basado en KNN utilizando la distancia euclidiana como metrica de similtud. Divide el dataset en conjunto de entrenamiento y prueba. Luego calcula el hit rate del modelo en el conjunto de prueba, explicando como el hit rate refleja la efectividad del recomendador en este caso

In [19]:


# Usar el algoritmo KNN con la métrica de distancia euclidiana para calcular similitudes entre ítems
sim_options = {
    'name': 'msd',  # Mean Squared Difference, equivalente a la distancia euclidiana
    'user_based': True  # Similitud basada en usuarios
}
algo4 = KNNBasic(sim_options=sim_options)

# Entrenar el modelo en el conjunto de entrenamiento
algo4.fit(trainset)



Computing the msd similarity matrix...
Done computing similarity matrix.


In [20]:
# Predecir las calificaciones en el conjunto de prueba
predictions = algo4.test(testset)



In [21]:
# Convertir predicciones a DataFrame
predictions_df = pd.DataFrame(predictions, columns=['userId', 'movieId', 'real_rating', 'predicted_rating', 'details'])


In [22]:
# Definir umbral de relevancia
relevant_threshold = 4.0

# Inicializar contadores
hits = 0
total_relevant = 0

# Iterar sobre todas las predicciones
for _, row in predictions_df.iterrows():
    real_rating = row['real_rating']
    predicted_rating = row['predicted_rating']

    # Considerar películas con calificación real ≥ umbral como relevantes
    if real_rating >= relevant_threshold:
        total_relevant += 1
        # Si la predicción también es ≥ umbral, es un acierto
        if predicted_rating >= relevant_threshold:
            hits += 1

# Calcular el Hit Rate Global
hit_rate_global = hits / total_relevant if total_relevant > 0 else 0
print(f"Hit Rate Global: {hit_rate_global:.2f}")

Hit Rate Global: 0.47


Conclusiones:

Un hit rate global de 0.47 significa que, en aproximadamente el 47% de las recomendaciones que el sistema hizo, la predicción fue exitosa: es decir, cuando el sistema predijo que una película sería apreciada por el usuario (con una calificación predicha de al menos 4.0), el usuario efectivamente le dio una calificación de 4.0 o superior.

Es un buen punto de partida, sin embargo para mejorarlo se tendría que hacer:
- Optimizar el modelo, con otras metricas de similitud, realizar ajuste de hiperparametros
- Optimizacion de la metrica, ajustando threshold = 4.0.
- Evaluación con más métricas: Incluir otras métricas, como el RMSE o el MAE